## Transforming NIH Reporter files into KGTK files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from glob import glob
import csv
import tqdm.notebook as tqdm

In [ ]:
from dateutil.parser import parse
def add_quote(a):
    return '"{0}"'.format(a)


paths = glob('/content/drive/MyDrive/Colab Notebooks/NIH/2021/*')

for i in (["2019"]):
  
  # num = str(i).split("FY")[1]
  # year = str(num.split(".csv")[0])
  year = i
  print("On FILE \n", year)
  data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NIH/2021/RePORTER_PRJ_C_FY" + str(year) + ".csv", encoding = "latin", error_bad_lines= False)

  organization_data = data[['ORG_IPF_CODE', 'ORG_NAME', 'ORG_CITY', 'ORG_STATE', 'ORG_ZIPCODE', 'ORG_COUNTRY', 'ORG_DUNS']]
  organization_data.drop_duplicates(keep = 'first', inplace = True)
  organization_data.set_index('ORG_IPF_CODE', drop = True, inplace = True)


  project = data[['APPLICATION_ID', 'CORE_PROJECT_NUM', 'PROJECT_TITLE', 'PHR', 'PROJECT_TERMS','FOA_NUMBER','IC_NAME', 'PROGRAM_OFFICER_NAME','PROJECT_START','PROJECT_END','TOTAL_COST', 'TOTAL_COST_SUB_PROJECT']]
  project.drop_duplicates(keep = 'first', inplace = True)
  project.set_index('APPLICATION_ID', drop = True, inplace = True)
        
  PI_ID = data['PI_IDS']
  pi_name = data['PI_NAMEs']
  ORG_IPF_code = data['ORG_IPF_CODE']
  required_dict = list(zip(PI_ID, pi_name, ORG_IPF_code))
  investigator_list = []
  # print(required_dict)

  for i in required_dict:
      if str('contact') in i[0]:
          ids = list(i[0].split(';'))[:-1]
          names = list(i[1].split(';'))[:-1]
          org = i[2]
          for i in range(len(ids)):
                  if 'contact' in ids[i]:
                      pi_id = ids[i].split('(')[0]
                      pi_names = names[i].split('(')[0]
                      contact = 1
                  else: 
                      pi_id = ids[i]
                      pi_names = names[i]
                      contact = 0
                  investigator_list.append([pi_id.strip(), pi_names.strip(), org, contact])
      else:
          pi_id = i[0].split(';')[0]
          pi_names = i[1].split(';')[0]
          org = i[2]
          contact = 1
          investigator_list.append([pi_id.strip(), pi_names.strip(), org, contact])
          
  investigator = pd.DataFrame(investigator_list, columns=['PI_ID','PI_NAME','ORG_IPF_CODE', 'CONTACT'])
  # print(len(investigator_list))
  investigator.drop_duplicates(keep = 'first', inplace = True)
  # print(investigator)
  investigator_data = investigator[['PI_NAME','ORG_IPF_CODE']]
  investigator_data.set_index(investigator['PI_ID'], drop = True, inplace = True)


  PI_ID = data['PI_IDS']
  ORG_IPF_code = data['ORG_IPF_CODE']
  pi_name = data['PI_NAMEs']
  application_id = data['APPLICATION_ID']
  required_dict = list(zip(PI_ID, ORG_IPF_code, pi_name, application_id))

  investigator_list = []
  # print(required_dict)

  for i in required_dict:
        if str('contact') in i[0]:
            ids = list(i[0].split(';'))[:-1]
            names = list(i[2].split(';'))[:-1]
            org = i[1]
            app = i[3]
            for i in range(len(ids)):
                    if 'contact' in ids[i]:
                        pi_id = ids[i].split('(')[0]
                        pi_names = names[i].split('(')[0]
                        contact = 1
                    else: 
                        pi_id = ids[i]
                        pi_names = names[i]
                        contact = 0
                    investigator_list.append([app, pi_id.strip(), org, pi_names.strip(), contact])
        else:
            pi_id = i[0].split(';')[0]
            pi_names = i[2].split(';')[0]
            org = i[1]
            contact = 1
            app = i[3]
            investigator_list.append([app, pi_id.strip(), org, pi_names.strip(), contact])

  has_investigator = pd.DataFrame(investigator_list, columns=['APPLICATION_ID','PI_ID','ORG_IPF_CODE','PI_NAME', 'contactPI'])
  # print(len(investigator_list))
  has_investigator.drop_duplicates(keep = 'first', inplace = True)
  # print(has_investigator)
  has_investigator_data = has_investigator[['APPLICATION_ID','PI_ID','contactPI']]
  has_investigator_data.drop_duplicates(keep = 'first', inplace = True)
  has_investigator_data.set_index(['APPLICATION_ID','PI_ID'], drop = True, inplace = True)
  # has_investigator_data

  organization.to_csv('/content/drive/Shareddrives/Harmonize/datasets/NIH_Reporter/sample/organization'  + str(year) + '.csv')
  project.to_csv('/content/drive/Shareddrives/Harmonize/datasets/NIH_Reporter/sample/project' + str(year) + '.csv')
  investigator_data.to_csv('/content/drive/Shareddrives/Harmonize/datasets/NIH_Reporter/sample/investigator' + str(year) + '.csv')
  has_investigator_data.to_csv('/content/drive/Shareddrives/Harmonize/datasets/NIH_Reporter/sample/hasInvestigator' + str(year) + '.csv')


  project = pd.read_csv("/content/drive/Shareddrives/Harmonize/datasets/NIH_Reporter/sample/project"+ str(year) + '.csv')
  investigator = pd.read_csv("/content/drive/Shareddrives/Harmonize/datasets/NIH_Reporter/sample/investigator"+ str(year) + '.csv')
  organization = pd.read_csv("/content/drive/Shareddrives/Harmonize/datasets/NIH_Reporter/sample/organization"+ str(year) + '.csv')
  hasInvestigator = pd.read_csv('/content/drive/Shareddrives/Harmonize/datasets/NIH_Reporter/sample/hasInvestigator'+ str(year) + '.csv')


  project.replace(float('NaN'), "", inplace = True)
  # print(len(project)) #.head()


  project.drop_duplicates(keep = "first", inplace = True)
  organization_data = organization_data.iloc[:, 1:]
  # organization.head()
  project.drop_duplicates(keep = "first", inplace = True)

  investigator = investigator_data
  investigator_data.reset_index(inplace = True)
  has_investigator_data.reset_index(inplace = True)
  organization_data.reset_index(inplace = True)

  investigator_data["ORG_IPF_CODE"].isnull().sum()

  has_investigator_data.isnull().sum()

  investigator_data.fillna('', inplace = True)
  has_investigator_data.fillna('', inplace = True)

  organization_data.fillna('', inplace = True)

  # Create prefix for qnodes of different types: person (investigators), 
  #  organization (universities) and project (grants)

  tuple_prefix = "Q" + "NIH" + "T"
  grant_prefix = "Q" + "NIH" + "G"
  human_prefix = "Q" + "NIH" + "H" 
  person_prefix = "Q" + "NIH" + "PER"
  org_prefix = "Q" + "NIH" + "ORG" 
  project_prefix = "Q" + "NIH" + "PRO"
  nih_id = "P" + "NIH" + "ID"


  # make person kgtk file containing PI_ID, PI_NAME AND ORG_IPF_CODE

  person_kgtk = pd.DataFrame(columns = ["id", "node1", "label", "node2"])
  # person_kgtk.to_csv("/content/drive/Shareddrives/Harmonize/datasets/NIH_Reporter/kgtk-files-nih" + "/nih_person" + "_" + str(year) + "n.tsv", mode = "a+", sep = "\t")

  print("INVESTIGATOR")
  for idx in tqdm.tqdm(range(len(investigator_data))):

    # print(investigator_data.loc[idx, "PI_ID"])
    if (investigator_data.loc[idx, "PI_NAME"]) != '' and investigator_data.loc[idx, "PI_ID"] != '':
      person_kgtk = person_kgtk.append({"id": "", "node1":  person_prefix + str(investigator_data.loc[idx, "PI_ID"]), "label": "label", "node2":  add_quote((investigator_data.loc[idx, "PI_NAME"]).strip())}, ignore_index = True)
      person_kgtk = person_kgtk.append({"id": "", "node1":  person_prefix + str(investigator_data.loc[idx, "PI_ID"]), "label": "PNIHID", "node2": add_quote((investigator_data.loc[idx, "PI_ID"]).strip())}, ignore_index = True)
    if investigator_data.loc[idx, "ORG_IPF_CODE"] != '' and investigator_data.loc[idx, "PI_ID"] != '':
      person_kgtk = person_kgtk.append({"id": "", "node1":  person_prefix + str(investigator_data.loc[idx, "PI_ID"]), "label": "P108", "node2":  org_prefix + str(int(investigator_data.loc[idx, "ORG_IPF_CODE"])).strip()}, ignore_index = True)
      person_kgtk = person_kgtk.append({"id": "", "node1":  org_prefix + str(int(investigator_data.loc[idx, "ORG_IPF_CODE"])).strip(), "label": nih_id, "node2":  add_quote(str(int(investigator_data.loc[idx, "ORG_IPF_CODE"])).strip()) }, ignore_index = True)
  person_kgtk = person_kgtk.set_index("id", drop = True)
  person_kgtk.to_csv("/content/drive/Shareddrives/Harmonize/datasets/NIH_Reporter/kgtk-files-nih" + "/nih_person" + "_" + str(year) + "new.tsv", mode = "a+", sep = "\t", quoting = csv.QUOTE_NONE)
  print("file_done person kgtk for", str(year))
  print(len(person_kgtk))

  # make investigator kgtk file containing APPLICATION_ID (project), AND PI_ID (person)

  investigator_kgtk = pd.DataFrame(columns = ["id", "node1", "label", "node2"])
  # investigator_kgtk.to_csv("/content/drive/Shareddrives/Harmonize/datasets/NIH_Reporter/kgtk-files-nih" + "/nih_investigator" + "_" + str(year) + ".tsv", mode = "a+", sep = "\t")

  print("HAS INVESTIGTOR")
  for idx in tqdm.tqdm(range(len(has_investigator_data))):

    if (has_investigator_data.loc[idx, "APPLICATION_ID"]) != '' and has_investigator_data.loc[idx, "PI_ID"] != '':
      investigator_kgtk = investigator_kgtk.append({"id": "", "node1":  project_prefix + str(has_investigator_data.loc[idx, "APPLICATION_ID"]), "label": "P8329", "node2":  person_prefix + str(has_investigator_data.loc[idx, "PI_ID"]).strip()}, ignore_index = True)

  investigator_kgtk = investigator_kgtk.set_index("id", drop = True)
  investigator_kgtk.to_csv("/content/drive/Shareddrives/Harmonize/datasets/NIH_Reporter/kgtk-files-nih" + "/nih_investigator" + "_" + str(year) + ".tsv", mode = "a+", sep = "\t", quoting = csv.QUOTE_NONE)

  print("done file investigator_kgtk for", str(year))
  print(len(investigator_kgtk))

  # make organization kgtk file containing ORG_IPF_CODE (org), AND PI_ID (person)

  state_label = "PNIHstate"
  city_label = "P159string"
  country_label = "P17string"

  organization_kgtk = pd.DataFrame(columns = ["id", "node1", "label", "node2"])
  # organization_kgtk.to_csv("/content/drive/Shareddrives/Harmonize/datasets/NIH_Reporter/kgtk-files-nih" + "/nih_organization" + "_" + str(year) + ".tsv", mode = "a+", sep = "\t")

  print("ORGANIZATION")
  for idx in tqdm.tqdm(range(len(organization_data))):

    if str(organization_data.loc[idx, "ORG_IPF_CODE"]) != '' and organization_data.loc[idx, "ORG_NAME"] != '':
      ipf_code = int(organization_data.loc[idx, "ORG_IPF_CODE"])
      organization_kgtk = organization_kgtk.append({"id": "", "node1":  org_prefix + str(ipf_code), "label": "label", "node2": add_quote((organization_data.loc[idx, "ORG_NAME"]).strip())}, ignore_index = True)
      if str(organization_data.loc[idx, "ORG_CITY"]).strip() != '':
        organization_kgtk = organization_kgtk.append({"id": "", "node1":  org_prefix + str(ipf_code), "label": "P159string", "node2":  add_quote((organization_data.loc[idx, "ORG_CITY"]).strip()) }, ignore_index = True)
      if str(organization_data.loc[idx, "ORG_STATE"]).strip() != '':
        organization_kgtk = organization_kgtk.append({"id": "", "node1":  org_prefix + str(ipf_code), "label": "PNIHstate", "node2":  add_quote((organization_data.loc[idx, "ORG_STATE"]).strip()) }, ignore_index = True)
      if str(organization_data.loc[idx, "ORG_COUNTRY"]).strip() != '':
        organization_kgtk = organization_kgtk.append({"id": "", "node1":  org_prefix + str(ipf_code), "label": "P17string", "node2":  add_quote((organization_data.loc[idx, "ORG_COUNTRY"]).strip())}, ignore_index = True)

  organization_kgtk = organization_kgtk.set_index("id", drop = True)
  organization_kgtk.to_csv("/content/drive/Shareddrives/Harmonize/datasets/NIH_Reporter/kgtk-files-nih" + "/nih_organization" + "_" + str(year) + ".tsv", mode = "a+", sep = "\t", quoting = csv.QUOTE_NONE)

  print("done file organization_kgtk for", str(year))
  print(len(organization_kgtk))

  organization_data
  

On FILE 
 2019


b'Skipping line 3706: expected 46 fields, saw 47\n'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py

INVESTIGATOR


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,



file_done person kgtk for 2019
256090
